In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime

In [4]:
# Add in stocks from the selected Portfolio
# stocks = ['^SPX','AMP','SCHW','BRK-B','AAPL','STLA','YUM','ADM','AMR','PWR','XOM','CE','ELV','DGX','REGN','MRK','AMAT','GOOG','JBL','ADI']
stocks = ['^SPX', 'INTC','AMD','NVDA']

In [5]:
# Define the time period
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=5*365)

In [6]:
# Initialize a dictionary to hold ticker data
stock_data = {}

# Fetch the monthly closing prices for each stock
for ticker in stocks:
    stock = yf.Ticker(ticker)
    df = stock.history(start=start_date, end=end_date, interval="1mo")
    stock_data[ticker] = df['Close']

# Combine the data into a single DataFrame
combined_data = pd.DataFrame(stock_data)

# Clean up the DataFrame by dropping rows with NaN values that can occur at the end
# combined_data.dropna(how='all', inplace=True)

In [7]:
combined_data.head()

,^SPX,INTC,AMD,NVDA
Date,,,,
2019-05-01 00:00:00-04:00,2752.060059,38.539875,27.410000,33.635017
2019-06-01 00:00:00-04:00,2941.760010,42.148087,30.370001,40.825199
2019-07-01 00:00:00-04:00,2980.379883,44.507744,30.450001,41.941349
2019-08-01 00:00:00-04:00,2926.459961,41.743080,31.450001,41.640564
2019-09-01 00:00:00-04:00,2976.739990,45.676945,28.990000,43.314117


In [8]:
# Calculate excess the month to month excess returns for each stock and store it in a new dataframe
excess_ret = combined_data.pct_change()
excess_ret.tail()

,^SPX,INTC,AMD,NVDA
Date,,,,
2023-12-01 00:00:00-05:00,0.044229,0.127857,0.216656,0.058841
2024-01-01 00:00:00-05:00,0.015896,-0.142686,0.137575,0.242527
2024-02-01 00:00:00-05:00,0.051721,-0.000696,0.148130,0.285809
2024-03-01 00:00:00-05:00,0.031019,0.029024,-0.062536,0.142128
2024-04-01 00:00:00-04:00,-0.046390,-0.220964,-0.176464,-0.119907


In [31]:
stocks.__len__()

3

In [10]:
# Add a row for total portfolio
for givenRow in excess_ret:
    
    excess_ret.loc[givenRow, "Portfolio"] = 

SyntaxError: invalid syntax (742023493.py, line 4)

In [19]:
# Optional // If a stock has NaN values find out where they begin
fst = combined_data['AMR'].first_valid_index()
ind = combined_data.index.get_loc(fst)
combined_data.iloc[(ind-3):(ind+3)]

KeyError: 'AMR'

In [9]:
# Separate the S&P 500 returns and the stocks' returns
spx_returns = excess_ret.iloc[:, 0]
stocks_returns = excess_ret.iloc[:, 0:]

# Create an empty DataFrame to store the analysis results
perf_metrics = pd.DataFrame(index=excess_ret.columns)

# Calculate each metric
perf_metrics['Mean Annualized Return'] = stocks_returns.mean() * 12
perf_metrics['Annualized Std Dev'] = stocks_returns.std() * np.sqrt(12)

In [10]:
perf_metrics

,Mean Annualized Return,Annualized Std Dev
^SPX,0.139073,0.183232
INTC,0.037418,0.350021
AMD,0.483930,0.537278
NVDA,0.781495,0.500194


In [11]:
# Market variance for Beta calculations
market_variance = spx_returns.var() * 12

# Calculate metrics that require row-wise operations
for stock in stocks_returns:
    stock_returns = stocks_returns[stock]
    cov_with_market = stock_returns.cov(spx_returns) * 12  # Annualize the covariance
    beta = cov_with_market / market_variance                                                                   
    
    perf_metrics.loc[stock, 'SPX Correlation'] = stock_returns.corr(spx_returns)
    perf_metrics.loc[stock, 'Beta'] = beta
    perf_metrics.loc[stock, 'Total Variance'] = stock_returns.var() * 12  # Annualize the variance         
    perf_metrics.loc[stock, 'Systematic Variance'] = beta ** 2 * market_variance                           
    perf_metrics.loc[stock, 'Unique Variance'] = perf_metrics.loc[stock, 'Total Variance'] - perf_metrics.loc[stock, 'Systematic Variance']
    perf_metrics.loc[stock, 'R-squared'] = perf_metrics.loc[stock, 'SPX Correlation'] ** 2 

# perf_metrics.format({})

In [12]:
perf_metrics       # what would you call this set of statistics? performance measures, can add sharpe ratio, sortino ratio, studdtzer index, maximum drawdown, sometimes called risk adjusted performance metrics, like sharpe ratio and  sortino this means they dont change with leverage

,Mean Annualized Return,Annualized Std Dev,SPX Correlation,Beta,Total Variance,Systematic Variance,Unique Variance,R-squared
^SPX,0.139073,0.183232,1.000000,1.000000,0.033574,0.033574,-1.387779e-17,1.000000
INTC,0.037418,0.350021,0.538553,1.028777,0.122515,0.035534,8.698072e-02,0.290039
AMD,0.483930,0.537278,0.584960,1.715234,0.288668,0.098776,1.898920e-01,0.342178
NVDA,0.781495,0.500194,0.618200,1.687587,0.250194,0.095617,1.545773e-01,0.382171


In [13]:
# Transpose the df
stks_anl_results_T = perf_metrics.transpose()
stks_anl_results_T

,^SPX,INTC,AMD,NVDA
Mean Annualized Return,1.390728e-01,0.037418,0.483930,0.781495
Annualized Std Dev,1.832320e-01,0.350021,0.537278,0.500194
SPX Correlation,1.000000e+00,0.538553,0.584960,0.618200
Beta,1.000000e+00,1.028777,1.715234,1.687587
Total Variance,3.357398e-02,0.122515,0.288668,0.250194
Systematic Variance,3.357398e-02,0.035534,0.098776,0.095617
Unique Variance,-1.387779e-17,0.086981,0.189892,0.154577
R-squared,1.000000e+00,0.290039,0.342178,0.382171
